# Desafio Prático de Machine Learning
***
> Solução feita por Alysson Machado de Oliveira Barbosa

### Instruções do Desafio
***

1. Escreva um algoritmo em Python que realize o treinamento de um modelo para a detecção de torres de energia e linhas de energia em imagens.
2. Siga as instruções do repositório para extração de rótulos no formato [COCO dataset](https://cocodataset.org/#home). Utilize imagens de tamanho 640 x 360.
3. Respeite a divisão da base de dados quanto aos conjuntos de treino, validação e teste.
4. Atente-se a um código limpo, organizado, documentado e com ideias claras da solução proposta.
5. Caso mais de um modelo seja produzido, comente sobre os resultados obtidos, explicitando qual deles funcionou melhor e o porquê.
6. Na falta de uma GPU para o treinamento, recomenda-se o uso do Kaggle ou do Google Colab

![imagem-capa](imagens-ilustrativas/imagem-capa.jpeg)
> Segmentação por instância de torres de energia e linhas de força.

## Abordagem do Problema
***

* ```Arquitetura Utilizada```: **YoloV8**.

A YOLOv8 é um modelo de rede neural utilizado para resolver problemas de segmentação por instância e detecção de objetos de forma simultânea. YOLO, que significa "*You Only Look Once*", é uma abordagem de detecção de objetos em tempo real que opera diretamente em uma imagem inteira, em vez de dividir a imagem em regiões menores, como outros métodos. O YOLOv8 é uma versão aprimorada que utiliza uma arquitetura de rede neural convolucional profunda para detectar e segmentar objetos em uma imagem, atribuindo rótulos e coordenadas de caixa delimitadora a cada objeto encontrado. Essa abordagem oferece uma detecção rápida e eficiente de objetos em tempo real, sendo amplamente utilizado em aplicações de vigilância, automação industrial, veículos autônomos e outras áreas onde a detecção de objetos é essencial.

* ```Servidor para treinar o modelo```: **Google Colab (GPU Tesla T4)**.

## Análise do Dataset
****

* ```Dataset utilizado```: [TTPLA: An Aerial-Image Dataset for Detection and Segmentation of Transmission Towers and Power Lines](https://github.com/R3ab/ttpla_dataset).

O dataset TTPLA contém imagens aéreas de torres e linhas de transmissão. Cada imagem possui anotações de segmentação poligonal dos objetos em análise, organizada em arquivos ```.json``` no formato [Coco Dataset](https://cocodataset.org/#home).

## Preparação dos Dados
***

#### 1. Redimensionamento das imagens

O dataset original foi obtido através do Google Drive, disponibilizado pelos criadores no GitHub, na qual o mesmo pode ser obtidos [clicando aqui](https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1Yz59yXCiPKS0_X4K3x9mW22NLnxjvrr0). Esse dataset foi adicionada a pasta ```dataset-original```. 

Em seguida, o script localizado em ```scripts/resize_image_and_annotation.py``` foi modificado para que fosse possível redimensionar as imagens do dataset para um tamanho de 640x360.   

A partir do script ```scripts/resize_image_and_annotation.py```, foi utilizado o seguinte código na linha de comando para que uma nova pasta fosse gerada com os códigos redimensionados:

```
python resize_image_and_annotation-final.py -t dataset-original/
```

A nova pasta com as imagens redimensionadas foi renomeada para ```dados-redimensionados/```.


#### 2. Remoção de dados rotulados em vazio

Para evitar que problemas fossem obtidos durante a etapa de treinamento, imagens que contêm regiões sem rótulo definido foram removidas. Foi utilizado o script localizado em ```scripts/remove_void.py``` para resolver esse problema, através do seguinte código de comando:

```
python remove_void.py -t dados-redimensionados/
```

A nova pasta com os arquivos ```.json``` corrigidos foi renomeada para ```jsons-corrigidos/```.

#### 3. Divisão do dataset em treinamento/validação/teste

Utilizando a sugestão de divisão dos dados, armazenados em arquivos ```.txt``` em estratos de treinamento, validação e teste, em uma aproximação respectiva de 70%, 10% e 20%, respectivamente, localizada na pasta ```guia-estratificacao-arquivos/```, o script ```scripts/split_jsons.py``` foi utilizado para resolver esse problema. O seguinte código de comando foi utilizado:

```
python split_jsons.py -t jsons-corrigidos/
```

A nova pasta com os arquivos ```.json``` estratificados foi renomeada para ```jsons-estratificados/```.

#### 4. Remoção de imagens com rotulações problemáticas

Para verificar se todas as imagens estão de fato no padrão [Coco Dataset](https://cocodataset.org/#home), para serem utilizadas no padrão de treinamento de uma arquitetura Yolo, por exemplo, o seguinte script localizado em ```scripts/labelme2coco_2.py``` foi utilizado para realizar essa verificação. O procedimento consistiu em modificar esse script definido pelos criadores do dataset, que tinha o objetivo primário de alterar os rótulos para o padrão da arquitetura ```YoloAct```, para que ele retornasse o caminho relativo da imagem que fazia o código não conseguir realizar essa conversão, de modo que o mesmo não fosse compilado devido a uma mensagem de erro. Dessa forma, todas as imagens problemáticas (em torno de 16 imagens) foram excluídas.

```
python labelme2coco_2.py jsons-estratificados/jsons-treinamento
python labelme2coco_2.py jsons-estratificados/jsons-validacao
python labelme2coco_2.py jsons-estratificados/jsons-teste
```

#### 5. Upload do dataset no RoboFlow para ser utilizado no Google Colab

Devido a limitações técnicas em meu computador pessoal, o Google Colab foi o servidor escolhido para treinar o modelo de detecção e segmentação por instância no YoloV8 utilizando o dataset pré-processado TTPLA. No Google Colab, uma GPU do tipos Tesla T4 foi utilizada.

O dataset pré-processado foi colocado no RoboFlow por três motivos:
1. Facilidade para baixá-lo utilizando código em Python pelo Google Colab;
2. Fácil reuso toda vez que uma nova sessão no servidor do Google Colab for criada;
3. Conversão fácil do dataset organizado no estilo do [Coco Dataset](https://cocodataset.org/#home) para o estilo do YoloV8;

O dataset pré-processado para cumprir com os requisitos do ICTS podem ser baixados pelo link: 
* [https://universe.roboflow.com/alysson-machado/ttpla-qx1sw](https://universe.roboflow.com/alysson-machado/ttpla-qx1sw).

![marcacao-exemplo1](imagens-ilustrativas/marcacao-exemplo1.jpeg)
> Imagen marcada e pré-processada no formato solicitado pelo desafio do ICTS.

![marcacao-exemplo2](imagens-ilustrativas/marcacao-exemplo2.jpeg)
> Imagen marcada e pré-processada no formato solicitado pelo desafio do ICTS.

## Preparação do Ambiente de Treinamento
***

In [1]:
# verificando a disponibilidade de GPU do Google Colab
!nvidia-smi

Sun Jun 18 18:04:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# verificando o diretório atual de trabalho do Google Colab
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
# instalando pelo pip a arquitetura da YoloV8
!pip install ultralytics==8.0.28
# limpando a célula do Jupyter com as informações de instalação do pacote
from IPython import display
display.clear_output()
# verificando o status da instalação do pacote e compatibilidade com o servidor
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.1/78.2 GB disk)


In [4]:
# importando a arquitetura YoloV8 do pacote ultralytics
from ultralytics import YOLO
from IPython.display import display, Image

## Baixando o Dataset Pré-processado no Servidor do Jupyter
***

In [5]:
# criando um diretório para baixar o dataset importado do roboflow
!mkdir {HOME}/datasets
%cd {HOME}/datasets

# instalando os pacotes do roboflow para fazer a conexão com a API
!pip install roboflow --quiet
from roboflow import Roboflow

# conectando com a API do RoboFlow para baixar o dataset pré-processado 
# OBS: por motivos didáticos, deixarei explicita a chave de acesso para a API 
# de download do dataset que fiz upload para o site, mas normalmente eu não constumo
# upar essa informação sensível em repositórios abertos
rf = Roboflow(api_key="xhUBoRw9sdlgzMz1Zw0Y")
project = rf.workspace("alysson-machado").project("ttpla-qx1sw")
dataset = project.version(1).download("yolov8")

/content/datasets
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.2 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.28, to fix: `pip install ultralytics<=8.0.20`

Extracting Dataset Version Zip to TTPLA-1 in yolov8:: 100%|██████████| 4780/4780 [00:02<00:00, 2204.34it/s]


## Treinando a Arquitetura da YoloV8
***

In [6]:
%cd {HOME}
# detalhes da linha de comando:
# 'task=segment' -> informar a arquitetura que os dados de treinamento estão organizados no padrão de segmentação
# 'mode=train' -> informar que a arquitetura pré-treinada vai ser retreinada utilizando transferência de aprendizado
# 'model=yolov8x-seg.pt' -> informar que a arquitetura a ser retreinada é do tipo x, da mais robusta
# 'data={dataset.location}/data.yaml' -> passa a informação da localização do dataset formatado
# 'epochs=30' -> define a quantidade de épocas de treinamento 
# 'imgsz=640' -> informa que a arquitetura a ser retreinada é para o padrão 640x360

!yolo task=segment mode=train model=yolov8x-seg.pt data={dataset.location}/data.yaml epochs=30 imgsz=640

/content
100% 137M/137M [00:07<00:00, 18.3MB/s]
Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=/content/datasets/TTPLA-1/data.yaml, epochs=30, patience=50, batch=16, imgsz=640, save=True, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False,

O modelo treinado foi adicionado a pasta ```modelo-treinado/```.

## Análise do Modelo Treinado
***

Diversos gráficos para os dados de treinamento e validação foram adicionadas a pasta ```metricas-avaliativas/``` para melhor análise. Nesses gráficos, é contemplado em mais detalhes a performace da capacidade de detecção de objetos e segmentação, utilizando métricas avaliativos como precisão, sensibilidade e F1-score.

![metricas](metricas-avaliativas/dados-treinamento/results.png)

Com base nos resultados obtidos, o modelo treinado apresenta alguns pontos fortes e pontos fracos. No lado positivo, o modelo alcança uma precisão (```precision```) relativamente alta para detecção de torres (0.85) e uma precisão moderada para detecção de linhas de energia (0.61). Além disso, a perda de caixa (```box loss```) e a perda de segmentação (```seg loss```) são consistentemente reduzidas durante o treinamento, sugerindo que o modelo está aprendendo a localizar e segmentar com eficiência esses elementos nas imagens aéreas.

Por outro lado, existem algumas áreas que podem ser melhoradas. Primeiramente, a perda de classificação (```cls loss```) e a perda de deslocamento de características (```dfl loss```) estão relativamente altas em comparação com as outras perdas. Isso pode indicar que o modelo está tendo dificuldade em classificar corretamente as torres e linhas de energia, bem como em realizar ajustes finos de localização. Além disso, as métricas de recall (```recall```) para ambos os objetos são baixas, indicando que o modelo está deixando de detectar algumas instâncias de torres e linhas de energia.

Para melhorar o modelo, algumas sugestões podem ser consideradas. Primeiramente, é recomendável aumentar o tamanho do conjunto de treinamento, adicionando mais exemplos de torres e linhas de energia em diferentes contextos e condições de iluminação. Isso ajudará o modelo a generalizar melhor para casos mais diversos. Além disso, ajustar os hiperparâmetros do modelo, como a taxa de aprendizado e o tamanho do lote, pode levar a melhorias adicionais no desempenho. Também pode ser útil explorar técnicas de aumento de dados, como rotações e espelhamentos, para fornecer ao modelo uma maior variedade de exemplos durante o treinamento. Por fim, realizar uma análise mais detalhada dos falsos positivos e falsos negativos durante a validação pode ser útil para melhorar a qualidade geral das detecções.

In [7]:
# listando as imagens com as métricas quantitativas e qualitativas obtidas a respeito do treinamento
# mais detalhes dessas métricas avaliativas estão disponíveis na pasta 'metricas-avaliativas/'
!ls {HOME}/runs/segment/train/

args.yaml					    train_batch0.jpg
BoxF1_curve.png					    train_batch1.jpg
BoxP_curve.png					    train_batch2100.jpg
BoxPR_curve.png					    train_batch2101.jpg
BoxR_curve.png					    train_batch2102.jpg
confusion_matrix.png				    train_batch2.jpg
events.out.tfevents.1687111713.32e5e3524560.1712.0  val_batch0_labels.jpg
MaskF1_curve.png				    val_batch0_pred.jpg
MaskP_curve.png					    val_batch1_labels.jpg
MaskPR_curve.png				    val_batch1_pred.jpg
MaskR_curve.png					    val_batch2_labels.jpg
results.csv					    val_batch2_pred.jpg
results.png					    weights


## Inferências nos Dados de Teste e em Vídeos
***

In [ ]:
# realizando uma predição em um vídeo de exemplo baixado do YouTube
%cd {HOME}
!yolo task=segment mode=predict model={HOME}/runs/segment/train/weights/best.pt conf=0.25 source='/content/video-example.mp4' save=true

In [ ]:
# realizando uma predição em um vídeo de exemplo baixado do YouTube
%cd {HOME}
!yolo task=segment mode=predict model={HOME}/runs/segment/train/weights/best.pt conf=0.25 source='/content/video-example2.mp4' save=true

In [ ]:
# realizando uma predição em um vídeo de exemplo baixado do YouTube
%cd {HOME}
!yolo task=segment mode=predict model={HOME}/runs/segment/train/weights/best.pt conf=0.25 source='/content/video-example3.mp4' save=true

In [ ]:
# realizando a predição nos dados selecionados como teste
%cd {HOME}
!yolo task=segment mode=predict model={HOME}/runs/segment/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=true

#### 1. Vídeos de Exemplo

A seguir, é possível analisar os resultados obtidos com a predição feita nos três vídeos selecionados. Os vídeos estão localizados na pasta ```resultados-videos/``` e também os coloquei no YouTube para maior facilidade de análise:

* Primeiro Vídeo: [https://youtu.be/QyB5Sq-Rt_A](https://youtu.be/QyB5Sq-Rt_A)
* Segundo Vídeo: [https://youtu.be/5booA-m-5Ns](https://youtu.be/5booA-m-5Ns)
* Terceiro Vídeo: [https://youtu.be/KNTggwp_2WI](https://youtu.be/KNTggwp_2WI)

#### 2. Imagens do Conjunto de Teste

A seguir, é possível visualizar o resultado de algumas imagens disponíveis no conjunto de teste. Mais imagens podem ser visualizadas na pasta ```resultados-teste/```.

|     |     |
| ----------- | ----------- |
| ![Imagem 1](resultados-teste/13_00263.rf.31bff92f2ab0cea9e341544b6749bbdc.jpg)  | ![Imagem 2](resultados-teste/16_3450.rf.2bdf90684515866182ec4c154579e982.jpg)  |
| ![Imagem 3](resultados-teste/19_00563.rf.00e9332d2073d6b790cfcdc05693d95d.jpg)  | ![Imagem 4](resultados-teste/19_00934.rf.54bf10bc96f149536fb23f9e7fa65df5.jpg)  |
| ![Imagem 5](resultados-teste/31_7785.rf.8a631e630477fea86c5d5ea8d32d1d45.jpg)  | ![Imagem 6](resultados-teste/33_7020.rf.e2d7ef6a43ad603718d17ef6e69df2a9.jpg)  |
